### Solver Pre-Compilation
Most of the time, using the 'algorithm =' argument within model.run() is the best practice for assigning a particular algorithm to be used when a particular model is ran. This however will create issues when the model is ran within a loop body as each iteration will take almost a full five seconds in make alone, dramatically increasing the time taken to run each iteration. This is why it is best to precompile any model within a loop body. Below is a quick example of how to do so. 

In [1]:
import sys
import os
import time
sys.path.insert(1, '../../')
import numpy
import matplotlib.pyplot as plt
import gillespy2

### Create a model

In [2]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='Substrate', initial_value=301)
            B = gillespy2.Species(name='Enzyme', initial_value=120)
            C = gillespy2.Species(name='Enzyme_Substrate_Complex', initial_value=0)
            D = gillespy2.Species(name='Product', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

### Instantiate the model without a solver or algorithm

In [3]:
model = MichaelisMenten()

### Call get_best_solver_algo on model object and pass the desired algorithm name like you would with 'algorithm ='. Also pass the model object in a seperate bracket. This will Precompile the model.

In [4]:
PrecompSolver = model.get_best_solver_algo("Tau-Hybrid")(model)

### Below is an example of why pre-compilation is neccsary when running a model in a loop body

In [5]:
start = time.perf_counter()
for i in range(20):
    ode_results = model.run(solver = PrecompSolver)
end = time.perf_counter()
Precomp = end - start

start = time.perf_counter()
for i in range(20):
    hybrid_results = model.run(algorithm = "Tau-Hybrid")
end = time.perf_counter() 
noPrecomp = end - start


In [7]:
print("Time to complete 20 runs using pre compiled model: " + str(Precomp))
print("Time to complete 20 runs without using pre compiled model: " + str(noPrecomp))

Time to complete 20 runs using pre compiled model: 0.35088030000000003
Time to complete 20 runs without using pre compiled model: 198.7868176
